# Сверточные нейронные сети


## Материал прошлой лекции

- функции активации

### Полносвязная нейронная сеть 
FCN Fully-connected Neural Net. 

В современных статьях чаще используют сокращение MLP (Multi Layer Perceptron)





На прошлом занятии мы рассотрели следующий подход:

- превращаем исходные данные в вектор 

В случае с цветным изображением 32x32 пикселя (32x32x3) этот Размер входного вектора будет равен 3072x1.

-  Перемножим(скалярно) этот вектор с матрицей весов.Размер матрицы весов может быть например 100x3072. Где 3072 - размер входа, а 100 количество признаков которые мы хотим получить. Результат будет иметь размер 100x1


- Поэлементно применяем к результату функцию обеспечивающую нелинейность, например Sigmoid или Relu (функция активации). Размерность данных при этом не меняется (100x1). Получили вектор активаций, или признаков.

- Используем признаки как входы для нового слоя. Количество весов слоя будет зависеть от размерности входа и того что мы хотим получить на выходе. Если мы делаем классификатор на 10 классов, то матрица весов должна иметь размерность 10x100 и на этом можно остановиться. Но в общем случае количество слоев может быть произвольным.


<img src ="http://edunet.kea.su/repo/src/L06_CNN/img/L06_1.png" width="700">



Что нам дало добавление 2-го слоя. Мы позволили модели использовать более одного шаблона на класс. Можно убедиться в этом визуализировав веса первого слоя обученной модели.


<img src ="http://edunet.kea.su/repo/src/L06_CNN/img/gan/mlp-templates.png" width="400">

И такая схема работает лучше чем один слой. Однако она не решает всех проблемм.
Сосредоточимся на изображениях.




## Нарушение связей между соседними пикселями

Что произойдет когда мы растягиваем изображение в вектор?

<img src ="http://edunet.kea.su/repo/src/L06_CNN/img/L06_9.png" width="700">

В результате такого преобразования теряется связь между соседними пикселями. 

А интуитивно понятно, что при обработке изображений важен принцип локальности. 

Набор темных пикселей например мог например составлять контур уха кошки. Но это можно понять только пока они находились рядом.

Нам необходимо найти способ как передать нейросети эту информацию. Идею для этого можно позаимствовать у лучшей в известном нам мире интеллектуальной системе - мозгу.

## Рецептивные поля

Наш мозг прианализе зрительных сбразов как раз учитывает локальную информацию.
Расположенные рядом на сетчатке рецепторные клетки (палочки и колбочки) связанны с одной нервной (биполярной клеткой).  

На следующим уровнях картина повторяется (одна ганглиозная клетка связанна с несколькими более простыми биполярными) и.т.д. 


Таким образом одна более сложная клетка обрабатывает сигнал с нескольких простых рецепторов расположенных рядом и возбуждается при определенной комбинации сигналов от них.

Область сетчатки которая контролируется нервной клеткой на определенном уровне называется ее рецептивным полем.




<img src ="http://edunet.kea.su/repo/src/L06_CNN/img/L06_10.png" width="700">

Итак, в контексте нейронной сети рецептивное поле определяется как размер области на входе, которая производит функцию. По сути, это мера связи выходного объекта (любого слоя) с входной областью (патчем). Важно отметить, что идея рецептивных полей применима к локальным операциям (например, свертка, объединение). Например, в задачах на основе движения, таких как прогнозирование видео и оценка оптического потока, мы хотим фиксировать большие движения (смещения пикселей в 2D-сетке), поэтому мы хотим иметь адекватное воспринимающее поле. В частности, воспринимающего поля должно быть достаточно, если оно больше, чем наибольшая величина потока в наборе данных. Конечно, есть много способов увеличить воспринимающее поле в CNN.

Заменить на что-то более близкое к HW

<img src ="http://edunet.kea.su/repo/src/L06_CNN/img/L06_11.png" width="700">

При использовании в этом смысле этот термин принимает значение, напоминающее рецептивные поля в реальных биологических нервных системах. CNN имеют особую архитектуру, имитирующую способ функционирования мозга реальных животных; вместо того, чтобы каждый нейрон в каждом слое соединялся со всеми нейронами в следующем слое ( многослойный перцептрон ), нейроны организованы в трехмерную структуру таким образом, чтобы учесть пространственные отношения между разными нейронами относительно исходные данные. Поскольку CNN используются в основном в области компьютерного зрения , данные, которые представляют нейроны, обычно являются изображениями; каждый входной нейрон представляет один пиксель исходного изображения. Первый слой нейронов состоит из всех входных нейронов; нейроны на следующем уровне будут получать соединения от некоторых входных нейронов (пикселей), но не от всех, как это было бы в случае MLP и других традиционных нейронных сетей. Следовательно, вместо того, чтобы каждый нейрон принимал соединения от всех нейронов на предыдущем уровне, CNN используют рецептивную полевую компоновку, в которой каждый нейрон получает соединения только от подмножества нейронов на предыдущем (нижнем) уровне. Рецептивное поле нейрона в одном из нижних слоев охватывает только небольшую область изображения, в то время как рецептивное поле нейрона в последующих (более высоких) слоях включает комбинацию рецептивных полей от нескольких (но не всех) нейронов в слой до (т.е. нейрон на более высоком уровне «смотрит» на большую часть изображения, чем нейрон на более низком уровне). Таким образом, каждый последующий слой способен изучать все более абстрактные особенности исходного изображения. Считается, что использование рецептивных полей таким образом дает CNN преимущество в распознавании визуальных паттернов по сравнению с другими типами нейронных сетей.

<img src ="http://edunet.kea.su/repo/src/L06_CNN/img/L06_12.png" width="700">

### Hubel & Wiesel,1959

<img src ="http://edunet.kea.su/repo/src/L06_CNN/img/L06_13.png" width="700">

### Ice Cube Model

Эта гипотетическая кубическая модель придумана для пояснения устройства клеток первичной визуальной коры, а именно  – как устроены предпочитаемые ориентации и, соответственно, реакции нейронов V1. Так, V1 можно условно поделить на кубы $2 мм^3$, каждый из которых получает сигналы от обоих глаз. Клетки с одинаковыми ориентационными предпочтениями формируют горизонтальные колонки, при этом соседние вертикальные колонки имеют слегка отличающиеся ориентационные предпочтения.

<img src ="http://edunet.kea.su/repo/src/L06_CNN/img/L06_14.png" width="700">

Чувствительные к цветам клетки также собраны в столбцы (также их называют каплями, гиперколонками, шариками) 0,5 мм в диаметре в зонах соответствующих превалирующих глаз (картинка с цилиндрами). Каждый такой столбец содержит реагирующие либо на красно-зеленый, либо на сине-желтый контрасты.

https://docplayer.ru/79903347-Kasyanov-evgeniy-dmitrievich-sankt-peterburg-nmic-pn-im-v-m-beh-k-m-n-fedotov-ilya-andreevich-ryazan-ryazgmu.html

## Примеры 'hancrafted' фильтров

### Фильтр Собеля

На заре компьютерного зрения люди придумывали различные операции, которые будут находить важные элементы на изображении. К примеру, границы. Примером подобной операции является фильтр Собеля. Он у Вас будет в практической работе.

<img src ="http://edunet.kea.su/repo/src/L06_CNN/img/sobel_example.png" width="300">

<img src ="http://edunet.kea.su/repo/src/L06_CNN/img/sobel.png" width="850">

##### $\color{brown}{\text{Допольнительная информация}}$

###### Детектор границ Canny

На самом деле, если мы будем применять данный алгоритм без подготовки изображения и дополнительных операций, то не получим хорошее качество детекции границ. Поэтому существуют детекторы границ, которые дополняют оператор Собеля (в алгоритме ниже оператор Собеля используется на шаге 2).

<img src ="http://edunet.kea.su/repo/src/L06_CNN/img/canny.png" width="850">

###### Детекторы углов

Другими примерами подобных вручную созданных детекторов признаков являются всевозможные детекторы углов. Одним из самых известных среди них является детектор углов Харриса. Мы не будем на нем останавливаться, но вы может о нем прочесть самостоятельно.

https://en.wikipedia.org/wiki/Harris_Corner_Detector


Как Вы увидите далее, сверточные нейронные сети, по факту автоматизируют процесс нахождения подобных признаков, высвобождая время исследователей и инженеров

Перемещая различные ядра свертки по изображению, можно выполнять манипуляции с изображениями. Вот как работают многие инструменты, которые можно найти в программах для редактирования изображений, таких как Photoshop или Gimp:

### Viola Jones ...

In [ ]:
import numpy as np
import scipy.misc
from skimage import data
import matplotlib.pyplot as plt
cat = data.chelsea().mean(axis = 2).astype('int32')

def im_show(arr):
  plt.figure()
  plt.imshow(arr, cmap='gray', vmin=0, vmax=255)

im_show(cat)
import torch
g_kernel5 = torch.tensor([
                     [1,4,7,4,1],
                     [4,16,26,16,4],
                     [7,26,41,26,7],
                     [4,16,26,16,4],
                     [1,4,7,4,1]]
                     ,dtype=torch.float) / 273 # sum of weights

gaussian_filter = nn.Conv2d(in_channels=1, out_channels=1, kernel_size=g_kernel5.shape, bias=False)
gaussian_filter.weight.data = g_kernel5.unsqueeze(0).unsqueeze(0) #gaussian_kernel
gaussian_filter.weight.requires_grad = False
#input = torch.tensor(cat).unsqueeze(0).unsqueeze(0).type(torch.float)
input = torch.tensor(cat).view(1,1,cat.shape[0],cat.shape[1]).type(torch.float)
out = gaussian_filter(input)
#print(out)
im_show(out[0][0].numpy())

## Свертка

Мы хотим что бы сеть сама научилась создавать новые фильтры. 

<img src ="http://edunet.kea.su/repo/src/L06_CNN/img/L06_9.png" width="700">

В

Вместо вектора весов размером с целое изображение используем квадратную матрицу небольшого размера.

<img src ="http://edunet.kea.su/repo/src/L06_CNN/img/gan/filter1.png" width="700">

### Свертка с фильтром

##### Изображение с одним каналом

В Deep Learning свертка - это поэлементное умножение и сложение. Для изображения с 1 каналом свертка показана на рисунке ниже. Здесь фильтр представляет собой матрицу 3 x 3 с элементом [[1, 0, 1], [0, 1, 0], [1, 0, 1]]. Фильтр скользит по входу. В каждой позиции он выполняет поэлементное умножение и сложение. Каждая скользящая позиция заканчивается одним номером. Окончательный результат - матрица 3 x 3.

<img src ="http://edunet.kea.su/repo/src/L06_CNN/img/L06-2.gif" width="300">

*Свертка изображения 5x5x1 с ядром 3x3x1 для получения свернутой функции 3x3x1*

В вышеприведенном демонстрации, зеленая секция напоминает наш входной 5x5x1 образ, я . Элемент, участвующий в выполнении операции свертки в первой части сверточного слоя, называется ядром / фильтром K , представленным желтым цветом. Мы выбрали K как матрицу 3x3x1.

#### Цветное/многоканальное изображение

В результате получим число, которое можно сравнить со значением нейрона. В нашем случае получится 10 значений. По этому принципу работают полносвязные слои.

Основное отличие свёрточных слоёв в том, что они сохраняют пространственную структуру изображения. Теперь мы будем использовать веса в виде небольших фильтров — пространственных матриц, которые проходят по всему изображению и выполняют скалярное произведение на каждом его участке. При этом **размерность фильтра (не путать с размером) всегда соответствует размерности исходного снимка**.
На самом деле почти всегда (так-как еще существуют depth-wise свертки. Но мы их сегодня рассматривать не будем.


<img src ="http://edunet.kea.su/repo/src/L06_CNN/img/conv1.png" width="500">

<img src ="http://edunet.kea.su/repo/src/L06_CNN/img/L06_15-1.png" width="200">

<img src ="http://edunet.kea.su/repo/src/L06_CNN/img/L06_15.png" width="700">

In [ ]:
import numpy as np

# let's do it manually

A = np.arange(9).reshape(3,3) # our matrix
C = np.array([[-1, 0], [0, 1]])

res = [[np.sum(A[:2, :2] * C), np.sum(A[:2, 1:3] * C)],
       [np.sum(A[1:3, :2] * C), np.sum(A[1:3, 1:3] * C)]]

print("A",A)
print("C",C)
print("conv",res)

Важно отметить, что каждый фильтр в слое считает от предыдущего слоя на всю глубину, но независмо от соседних с ним других фильтров в своем слое.

<img src ="http://edunet.kea.su/repo/src/L06_CNN/img/depth.jpeg" width="400">


*Свёрточный слой является основным слоем при построении свёрточных нейронных сетей.*

Обзор без погружения в особенности работы головного мозга. Давайте сперва попробуем разобраться в том, что же всё-таки вычисляет CONV-слой без погружения и затрагивания темы мозга и нейронов. Параметры свёрточного слоя состоят из набора обучаемых фильтров. Каждый фильтр представляет собой небольшую сетку вдоль ширины и высоты, но простирающуюся по всей глубине входного представления.

Например, стандартный фильтр на первом слое свёрточной нейронной сети может иметь размеры 5х5х3 (5px — ширина и высота, 3 — количество цветовых каналов). Во время прямого прохода мы перемещаем (если быть точными — свёртываем) фильтр вдоль ширины и высоты входного представления и вычисляем скалярное произведение между значениями фильтра и подлежащими значениями входного представления в любой точке. В процессе перемещения фильтра вдоль ширины и высоты входного представления мы формируем 2х мерную карту активации, которая содержит значения применения данного фильтра к каждой из областей входного представления. Интуитивно становится ясно, что сеть обучит фильтры активироваться при виде определённого визуального признака, например, прямой под определённым углом или колесообразных представлений на более высоких уровнях. Теперь, когда мы применили все наши фильтры к исходному изображению, например, их было 12. В результате применения 12 фильтров мы получили 12 активационных карт размерностью 2. Чтобы произвести выходное представление — объединим эти карты (последовательно по 3-му измерению) и получим представление размерностью [WxHx12].

Оценим количество праметров и количество операций ...

### Сверточный слой = Сверка + активация

В результате прохода по изображению мы получаем карту активации, также известную как карта признаков. Этот процесс называется пространственной свёрткой — более подробно о нём можно почитать в статье Свёртка в Deep Learning простыми словами. Из неё вы также можете узнать, почему размер карты активации получается меньше, чем у исходной фотографии.

К изображению можно применять множество фильтров и получать на выходе разные карты активации. Так мы сформируем один свёрточный слой. Чтобы создать целую нейросеть, слои чередуются друг за другом, а между ними добавляются функции активации (например, ReLU) и специальные pooling-слои, уменьшающие размер карт признаков.

<img src ="http://edunet.kea.su/repo/src/L06_CNN/img/gan/conv_layer.png" width="700">

- Сколько весов у такгого слоя?
- А сколько операций умножения потребуется сделать при прямом проходе?

А для FC - сети?

<img src ="http://edunet.kea.su/repo/src/L06_CNN/img/L06_1.png" width="700">


FC 3x32x32 = 3072 -> 512 
512->128
128->10

W = 3072 * 512 (1 572 864) + 512* 128 (65536) + 128* 10 (1280)  = 1639680 ~= 6.25 Mb (10% от датасета!)
FLOPS = 1.64M


Conv Layer

3x32x32 x 10x3x3F -> 6x32x32 x 128x3x3F-> 128x32x32 -> 512x3x3F

W = (3x3x3)*10+10 = 300 
+
(3x3x3)*128 +128 = 3584
+
(3x3x3)*512 +512 = 14366
= 18220 ~= 71Kb

FLOPS = 32*32*3*3*3*10 = 276 480
+
32*32*3*3*3*128 = 3 538 944
+
32*32*3*3*512 = 14 155 776

= 17 971 200 ~= 18M


Свертки содержат значительно меньшее количество обучаемых параметров. Но требуют больше вычислительных ресурсов (GPU)

### Параметры сверточного слоя

Обратите внимание, что некоторые слои содержат параметры, а некоторые — нет. В частности, CONV/FC-слои осуществляют трансформацию, которая является не только функцией зависящей от входных данных, но и зависящей от внутренних значений весов и смещений в самих нейронах. С другой стороны, RELU/POOL-слои применяют непараметризованные функции. Параметры в CONV/FC-слоях будут натренированы градиентным спуском таким образом, чтобы входные данные получали соответствующие корректные выходные метки.

Параметры:
- Размер фильтра (F)
- Шаг (stride)

<img src ="http://edunet.kea.su/repo/src/L06_CNN/img/L06_18.png" width="200">

Размер следующего слоя 

**N = 7, F = 3**

stride 1 => (7 - 3)/1 + 1 = 5 

stride 2 => (7 - 3)/2 + 1 = 3 

stride 3 => (7 - 3)/3 + 1 = **2.33** В данном случае часть изображения в свертку не попадет

<img src ="http://edunet.kea.su/repo/src/L06_CNN/img/L06_19.png" width="250">

In [ ]:
import torch
#input = torch.ones((1,1,7,7))
input = torch.tensor([[[[1,1,1,1,1,1,99],
                     [1,1,1,1,1,1,99],
                     [1,1,1,1,1,1,99],
                     [1,1,1,1,1,1,99],
                     [1,1,1,1,1,1,99],
                     [1,1,1,1,1,1,99]]]],dtype=torch.float)
print(input)
conv = torch.nn.Conv2d(in_channels=1, out_channels=1, kernel_size=3, stride = 3)
activation = conv(input) 
print(activation)

### Изменение размера за счет краев

<img src ="http://edunet.kea.su/repo/src/L06_CNN/img/L06_20.png" width="900">

### Расширение (padding)  

Можно заметить, что применение операции свертки уменьшает изображение. Также пиксели, которые находятся на границе изображения участвуют в меньшем количестве сверток, чем внутренние. В связи с этим в сверточных слоях используется дополнение изображения (англ. padding). Выходы с предыдущего слоя дополняются пикселями так, чтобы после свертки сохранился размер изображения. Такие свертки называют одинаковыми (англ. same convolution), а свертки без дополнения изображения называются правильными (англ. valid convolution). Среди способов, которыми можно заполнить новые пиксели, можно выделить следующие:

<img src ="http://edunet.kea.su/repo/src/L06_CNN/img/L06_21.png" width="250">

padding = (F-1)/2


*F - размер фильтра*

zero shift: 00[ABC]00;

border extension: AA[ABC]CC;

mirror shift: BA[ABC]CB;

cyclic shift: BC[ABC]AB.

In [ ]:
print(input)
conv = torch.nn.Conv2d(in_channels=1, out_channels=1, kernel_size=3, stride = 3,padding = int(3-1/2),padding_mode='zeros')
activation = conv(input) 
print(activation)

#### Входы и выходы сверточного слоя

<img src ="http://edunet.kea.su/repo/src/L06_CNN/img/gan/conv_sum.png" width="700">

<img src ="http://edunet.kea.su/repo/src/L06_CNN/img/L06-3.gif" width="700">



### Операция свертки в PyTorch

<img src ="http://edunet.kea.su/repo/src/L06_CNN/img/L06_23.png" width="700">

<img src ="http://edunet.kea.su/repo/src/L06_CNN/img/conv2_2.png" width="700">
<img src ="http://edunet.kea.su/repo/src/L06_CNN/img/conv2_3.png" width="700">

#### Рецептивные поля



<img src ="http://edunet.kea.su/repo/src/L06_CNN/img/gan/r-field.png" width="700">

Уменьшение размеров слоев 

- <font color=red >Не использовать расширение (padding)</font> 
- <font color=red >Увеличить шаг</font> 
- <font color=green >Свертка с фильтром 1x1</font> 
- <font color=green >Уплотнение (Субдискретизация, Pooling)</font> 

#### Свертка с фильтром 1x1

<img src ="http://edunet.kea.su/repo/src/L06_CNN/img/L06_24.png" width="700">

#### Уплотнение (Субдискретизация, Pooling)

<img src ="http://edunet.kea.su/repo/src/L06_CNN/img/L06_24_1.png" width="700">

#### Max and average pooling

<img src ="http://edunet.kea.su/repo/src/L06_CNN/img/L06_25.png" width="700">

<img src ="http://edunet.kea.su/repo/src/L06_CNN/img/gan/pool_sum.png" width="700">

### Другие виды сверток

###1D

Свертка - это математическая операция, которая объединяет две функции в третью. Например, допустим, что у нас есть две данные функции, **е(т)** а также **г(т)** и мы заинтересованы в применении одного поверх другого и вычислении площади пересечения:**f (t) * g (t) = (f * g) (t)**

В этом случае мы применяем **г(т)**(называется ядром) более **е(т)** и изменение ответа **(Е* г) (т)** согласно пересечению над областью обеих функций. Эта концепция свертки является наиболее используемой техникой обработки сигналов, поэтому она также применяется в компьютерном зрении, что можно рассматривать как обработку сигнала от нескольких датчиков RGB. Интуиция в эффективности свертки заключается в их способности фильтровать заданные входные сигналы в объединенный, более полезный результат.

В конкретном случае изображений сигнал лучше понимается с точки зрения матриц, а не форм волны. Поэтому наши функциие **(т)а** также **г(т)** теперь станетизображение *(матрица)* а также Ядро *(матрица)* соответственно. 
Еще раз, мы можем применить свертку, сдвинув одну из матриц поверх другой:

<img src ="http://edunet.kea.su/repo/src/L06_CNN/img/L06.gif" width="700">

<img src ="http://edunet.kea.su/repo/src/L06_CNN/img/L06-1.png" width="300">

То, что вы видите выше, это просто матричное умножение скользящего окна на изображении с ядром, а затем сложение суммы. Сила сверток в контексте Computer Vision заключается в том, что они являются отличными экстракторами функций для области датчиков RGB. При индивидуальном рассмотрении каждый пиксель (датчик RGB) бессмысленно понимать, что содержит изображение. Именно отношения друг с другом в пространстве придают образу истинный смысл. Это относится к тому, как вы читаете эту статью на своем компьютере, когда пиксели, представляющие символы, и ваш мозг сопоставляют черные пиксели друг с другом в пространстве, чтобы сформировать концепцию символов.

https://pytorch.org/docs/stable/generated/torch.nn.Conv1d.html

In [ ]:
import torch
from torch import nn
#torch.nn.Conv1d(in_channels, out_channels, kernel_size, stride=1, padding=0, dilation=1, groups=1, bias=True, padding_mode='zeros')

m = nn.Conv1d(16, 33, 3, stride=2)
input = torch.randn(20, 16, 50)
output = m(input)


### 3D

<img src ="http://edunet.kea.su/repo/src/L06_CNN/img/gan/3d_conv.png" width="700">



In [ ]:
#torch.nn.Conv3d(in_channels, out_channels, kernel_size, stride=1, padding=0, dilation=1, groups=1, bias=True, padding_mode='zeros')

# With square kernels and equal stride
m = nn.Conv3d(16, 33, 3, stride=2)
# non-square kernels and unequal stride and with padding
m = nn.Conv3d(16, 33, (3, 5, 2), stride=(2, 1, 1), padding=(4, 2, 0))
input = torch.randn(20, 16, 10, 50, 100)
output = m(input)
print(output.shape)

##### $\color{brown}{\text{Допольнительная информация}}$

###### Реализация

Операция свертки может быть реализованна как скалярное произведение.

<img src ="http://edunet.kea.su/repo/src/L06_CNN/img/gan/cm1.png" width="700">

<img src ="http://edunet.kea.su/repo/src/L06_CNN/img/gan/cm2.png" width="700">

<img src ="http://edunet.kea.su/repo/src/L06_CNN/img/gan/cm3.png" width="700">

<img src ="http://edunet.kea.su/repo/src/L06_CNN/img/gan/cm4.png" width="700">

<img src ="http://edunet.kea.su/repo/src/L06_CNN/img/gan/cm5.png" width="700">

<img src ="http://edunet.kea.su/repo/src/L06_CNN/img/gan/cm6.png" width="700">






https://medium.com/@_init_/an-illustrated-explanation-of-performing-2d-convolutions-using-matrix-multiplications-1e8de8cd2544

###### Справочник по сверткам

https://arxiv.org/pdf/1603.07285v1.pdf

https://towardsdatascience.com/types-of-convolutions-in-deep-learning-717013397f4d


the best: 
https://towardsdatascience.com/a-comprehensive-introduction-to-different-types-of-convolutions-in-deep-learning-669281e58215


## Сверточная сеть

<img src ="http://edunet.kea.su/repo/src/L06_CNN/img/gan/cnn.png" width="900">

#### LeNet

<img src ="http://edunet.kea.su/repo/src/L06_CNN/img/gan/lenet5.png" width="700">

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class Net(nn.Module):

    def __init__(self):
        super(Net, self).__init__()
        # 1 input image channel, 6 output channels, 3x3 square convolution
        # kernel
        self.conv1 = nn.Conv2d(1, 6, 3)
        self.conv2 = nn.Conv2d(6, 16, 3)
        # an affine operation: y = Wx + b
        self.fc1 = nn.Linear(16 * 6 * 6, 120)  # 6*6 from image dimension
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        # Max pooling over a (2, 2) window
        x = F.max_pool2d(F.relu(self.conv1(x)), (2, 2))
        # If the size is a square you can only specify a single number
        x = F.max_pool2d(F.relu(self.conv2(x)), 2)
        x = torch.flatten(x,1) 
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x


net = Net()

In [ ]:
print(net)

It is a simple feed-forward network. It takes the input, feeds it through several layers one after the other, and then finally gives the output.

A typical training procedure for a neural network is as follows:

* Define the neural network that has some learnable parameters (or weights)
* Iterate over a dataset of inputs
* Process input through the network
* Compute the loss (how far is the output from being correct)
* Propagate gradients back into the network’s parameters
* Update the weights of the network, typically using a simple update rule: weight = weight - learning_rate * gradient

https://pytorch.org/tutorials/beginner/blitz/neural_networks_tutorial.html

Let’s try a random 32x32 input. Note: expected input size of this net (LeNet) is 32x32. To use this net on the MNIST dataset, please resize the images from the dataset to 32x32.

In [ ]:
input = torch.randn(1, 1, 32, 32)
out = net(input)
print(out)

Код для обучения не отличается от FC сети

In [ ]:
import torch.optim as optim

# create your optimizer
optimizer = optim.SGD(net.parameters(), lr=0.01)

criterion = nn.CrossEntropyLoss().cuda()

target = torch.tensor([1])  # a dummy target, for example
# in your training loop:
optimizer.zero_grad()   # zero the gradient buffers
output = net(input)

loss = criterion(output, target)
loss.backward() # backprop
optimizer.step()    # Does the update

# Визуализация сетей

<img src ="http://edunet.kea.su/repo/src/L06_CNN/img/L06-5.png" width="700">

In [ ]:
from torchvision import models,utils
import matplotlib.pyplot as plt
import numpy as np

plt.rcParams["figure.figsize"] = (25,25)

alexnet = models.alexnet(pretrained=True)
print(alexnet)
weight_tensor = alexnet.features[0].weight.data
print(weight_tensor.shape)

img_grid = utils.make_grid(weight_tensor,pad_value=10)
plt.imshow(np.transpose(img_grid , (1, 2, 0)))



<img src ="http://edunet.kea.su/repo/src/L06_CNN/img/L06-6.png" width="700">

<img src ="http://edunet.kea.su/repo/src/L06_CNN/img/L06-7.png" width="700">

<img src ="http://edunet.kea.su/repo/src/L06_CNN/img/L06-8.png" width="700">

<img src ="http://edunet.kea.su/repo/src/L06_CNN/img/L06-9.png" width="700">

### Visualizing Live Convnet Activations

Построение значений активации нейронов в каждом слое сети в ответ на изображение или видео. В полностью связанных нейронных сетях порядок единиц не имеет значения, именно поэтому графики этих векторов не являются пространственно информативными. Однако в сверточных сетях фильтры применяются таким образом, чтобы учитывать лежащую в основе геометрию входных данных. 

В случае же двумерных изображений фильтры применяются в ДВУМЕРНОЙ свертке по двум пространственным измерениям изображения. Эта свертка производит активации на последующих слоях, которые для каждого канала также расположены пространственно.

http://yosinski.com/media/papers/Yosinski__2015__ICML_DL__Understanding_Neural_Networks_Through_Deep_Visualization__.pdf

<img src ="http://edunet.kea.su/repo/src/L06_CNN/img/L06-12.png" width="700">

<img src ="http://edunet.kea.su/repo/src/L06_CNN/img/L06-13.png" width="700">

### Активация слоев

Наиболее очевидный метод визуализации заключается в том, чтобы показать активации сети во время прямого прохода. Для сетей ReLU активации обычно начинает выглядеть относительно сгущенной и плотной, но по мере развития обучения активации обычно становится более редкой и локализованной. 

Единственная опасная ловушка, которую можно легко заметить с помощью этой визуализации, заключается в том, что некоторые карты активации могут быть равны нулю для многих различных входов. Это может указывать на мертвые фильтры и может быть симптомом высокой скорости обучения.

<img src ="http://edunet.kea.su/repo/src/L06_CNN/img/L06-14.png" width="700">

#### Окклюзирующие части изображения
Предположим, что CNN относит изображение к классу "собака". Как мы можем быть уверены, что он действительно улавливает собаку на изображении, а не контекстуальные сигналы с фона или другого объекта? Один из способов исследовать, из какой части изображения исходит некоторое классификационное предсказание, заключается в построении графика вероятности интересующего класса (например, класса собак) в зависимости от положения объекта окклюдера. 

То есть мы перебираем области изображения, устанавливаем патч изображения равным нулю и смотрим на вероятность класса. Мы можем визуализировать вероятность как 2-мерную тепловую карту. Этот подход был использован в визуализации и понимании сверточных сетей Мэтью Зейлера.

https://cs231n.github.io/understanding-cnn/

<img src ="http://edunet.kea.su/repo/src/L06_CNN/img/L06-15.png" width="700">

<img src ="http://edunet.kea.su/repo/src/L06_CNN/img/L06-16.png" width="700">

<img src ="http://edunet.kea.su/repo/src/L06_CNN/img/L06-17.png" width="700">

<img src ="http://edunet.kea.su/repo/src/L06_CNN/img/L06-18.png" width="700">

<img src ="http://edunet.kea.su/repo/src/L06_CNN/img/L06-19.png" width="700">

<img src ="http://edunet.kea.su/repo/src/L06_CNN/img/L06-20.png" width="700">

<img src ="http://edunet.kea.su/repo/src/L06_CNN/img/L06-21.png" width="700">

<img src ="http://edunet.kea.su/repo/src/L06_CNN/img/L06-22.png" width="700">

<img src ="http://edunet.kea.su/repo/src/L06_CNN/img/L06-23.png" width="700">

## Визуализация Модели класса

Далее разберем методику визуализации моделей классов, изученную с помощью сверточных нейронных сетей. Этот метод визуализации состоит в численном генерировании изображения, которое является репрезентативным для класса CNN. (which is representative of the class in terms of the ConvNet class scoring model.) 

Пускай Sc(I) - оценка класса c, вычисленная классификационным слоем CNN для изображения I. Мы хотели бы найти регуляризованное изображение L2, таким, чтобы показатель Sc высоким:

<img src ="http://edunet.kea.su/repo/src/L06_CNN/img/1-1.png" width="500">

где λ - параметр регуляризации. Локально-оптимальный показатель I может быть найден методом обратного распространения. Этот процесс напрямую связан с обучением CNN, где обратное распространение используется для оптимизации весов по слоям. Разница в том, что в нашем случае оптимизация выполняется по отношению к входному изображению, а веса фиксируются к тем, которые были найдены на этапе обучения.
Мы инициализировали оптимизацию с нулевым изображением (в нашем случае CNN была обучена на данных изображения с нулевым центром), а затем добавили к результату среднее изображение обучающего набора.

Стоит также отметить, что мы использовали (ненормализованные) оценки классов Sc, а не апостериоры классов, возвращаемые слоем soft-max:

<img src ="http://edunet.kea.su/repo/src/L06_CNN/img/1-2.png" width="300">

Причина в том, что максимизация класса posterior может быть достигнута за счет минимизации баллов других классов. Таким образом, мы оптимизируем Sc для того, чтобы убедиться в том, что оптимизация концентрируется только на рассматриваемом классе Sc. 

https://arxiv.org/pdf/1312.6034.pdf

<img src ="http://edunet.kea.su/repo/src/L06_CNN/img/L06-24.png" width="700">

<img src ="http://edunet.kea.su/repo/src/L06_CNN/img/L06-25.png" width="700">

<img src ="http://edunet.kea.su/repo/src/L06_CNN/img/L06-26.png" width="700">

<img src ="http://edunet.kea.su/repo/src/L06_CNN/img/L06-27.png" width="700">

<img src ="http://edunet.kea.su/repo/src/L06_CNN/img/L06-28.png" width="700">

<img src ="http://edunet.kea.su/repo/src/L06_CNN/img/L06-29.png" width="700">

<img src ="http://edunet.kea.su/repo/src/L06_CNN/img/L06-30.png" width="700">

<img src ="http://edunet.kea.su/repo/src/L06_CNN/img/L06-31.png" width="700">

<img src ="http://edunet.kea.su/repo/src/L06_CNN/img/L06-32.png" width="700">

## Практика

#### Pytorch. Tensor

<img src ="http://edunet.kea.su/repo/src/L06_CNN/img/L06_28.png" width="700">

https://pytorch.org/tutorials/beginner/blitz/neural_networks_tutorial.html